# Importance weighted autoencoder in Tensorflow

In [1]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
batch_size = 128
y_dim = 10

In [5]:
Y = tf.placeholder(tf.float32, [batch_size, y_dim])

In [8]:
yb = tf.reshape(Y, tf.stack([batch_size, 1, 1, y_dim]))

In [23]:
sess = tf.Session()

batch_size = 3
X = tf.constant(0.1, shape=[batch_size, 1920, 1080, 4])
w = tf.constant(0.1, shape=[10, 5, 4, 100])

h1 = tf.nn.conv2d(X, w, strides=[1, 10, 5, 1], padding='SAME')
sess.run(h1).shape

(3, 192, 216, 100)

In [5]:
def xavier_init(fan_in, fan_out, constant=1):
    """Xaiver initialization of network weights
    """
    low = -constant*np.sqrt(6.0/(fan_in + fan_out))
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out),
                            minval=low, maxval=high,
                            dtype=tf.float32)
    

In [ ]:
class VariationalAutoencoder(object):
    """Variatioinal Autoencoder with an sklearn-like interface implemented using TensorFlow.
    """
    def __init__(self, network_architecture, transfer_fct=tf.nn.softplus,
                 learning_rate=0.001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        slef.learning_rate = learning_rate
        self.batch_size = batch_size
        
        # tf.Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]])
        
        # Create autoencoder network
        self._create_network()
        
        # 変分上限に基づくロス関数それに対するオプティマイザを定義
        self._create_loss_optimizer()
        
        # tensorflow 変数の初期化関数
        init = tf.global_variables_initializer()
        
        # sessionの開始
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
        
    def _create_network(self):
        network_weights = self._initialize_weights(**self.network_architecture)
        
        self.z_mean, slef.z_log_sigma_sq = \
            self._recognition_network(network_weights['weights_recog'],
                                      network_weights['biases_gener'])
    
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_22,
                            n_hidden_gener_1, n_hidden_gener_2,
                            n_input, n_z):
        all_weights = dict()
        all_weights['weights_recog'] = {
            'h1' : tf.Variable(xaiver_init(n_input, n_hidden_recod_1)),
            'h2' : tf.Variable(xaiver_init(n_hidden_recog_1, n_hidden_recog_2)),
            'out_mean' : tf.Variable(xaaiver_init(n_hidden_recog_2, n_z)),
            'out_log_sigma' : tf.Vairable(xaiver_init(n_hidden_recog_2, n_z))
        }
        all_weights['biases_recog'] = {
            'b1' : tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
            'b2' : tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
            'out_mean' : tf.Variable(xaiver)
        }
    